In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Functions
##### Includes functions for normality tests and plots, z-score calculation and z-score plots, and comparison plots between length of chromosomes and number of miRNAs

In [2]:
# Function to convert chromosome length in bp to Mbp
def convert_to_megabase(df, bp_column='Length (bp)'):
    df['Length (Mbp)'] = df[bp_column] / 1_000_000
    return df

In [3]:
# Function to calculate miRNA density, taking into account chromosome length
# For example, a chromosome might have lesser microRNAs only because it is smaller, and might actually have a higher miRNA density
def calculate_mirna_density(df, mirna_column='Number of miRNA', length_column='Length (Mbp)'):
    df['miRNA_density'] = df[mirna_column] / df[length_column]
    return df

In [4]:
# Function to test for normality of the data using Shapiro-Wilk test
def run_shapiro_tests(df_list, column):
    results = {}
    for df, organism in zip(df_list, organisms):
        stat, p = stats.shapiro(df[column])
        results[organism] = {'W-statistic': stat, 'p-value': p}
    return results

In [5]:
# Functions to visualize the normality of the miRNA density and save images in figures with subplots for each organism
def plot_all_histograms(df_list, organisms):
    # Create a subplot figure with 6 subplots (2 rows, 3 columns)
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()  # Flatten to easily loop through the subplots
    for i, (df, org) in enumerate(zip(df_list, organisms)):
        sns.histplot(df['miRNA_density'], kde=True, ax=axes[i])
        axes[i].set_title(f'{org}: Histogram of miRNA Density')
        axes[i].set_xlabel('miRNA Density')
        axes[i].set_ylabel('Frequency')
    plt.tight_layout()
    plt.savefig('images/all_histograms.png')  # Save combined figure
    plt.close() # Close the plot to avoid memory issues

def plot_all_qq(df_list, organisms):
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    for i, (df, org) in enumerate(zip(df_list, organisms)):
        stats.probplot(df['miRNA_density'], dist="norm", plot=axes[i])
        axes[i].set_title(f'{org}: Q-Q Plot of miRNA Density')
    plt.tight_layout()
    plt.savefig('images/all_qqplots.png')  # Save combined figure
    plt.close()

In [6]:
# Function for calculating z scores to test for statistically significant deviations from the mean
# Z-score assumes data is normally distributed (which it is from normality tests)
def calculate_z_scores(df_list, mirna_density_column='miRNA_density'):
    for df in df_list:
        # Calculate the z-score for miRNA density
        df['z_score'] = stats.zscore(df[mirna_density_column])
        
        # Mark outliers based on z-score threshold (absolute value greater than 3)
        df['outlier'] = df['z_score'].abs() > 3
    return df_list  # Return the updated list of dataframes

In [7]:
# Function to plot individual z-scores for each organism and save plots
def plot_z_scores(df_list, organisms):
    for df, organism in zip(df_list, organisms):
        plt.figure(figsize=(12, 6))
        
        # Plot z-scores for each organism's data
        sns.barplot(data=df, x='Chromosome', y='z_score', hue='outlier', palette='coolwarm')
        
        # Add horizontal lines to indicate z-score threshold for outliers
        plt.axhline(0, color='gray', linestyle='--')
        plt.axhline(3, color='red', linestyle='--')
        plt.axhline(-3, color='red', linestyle='--')

        # Set plot title and labels
        plt.title(f'{organism}: Z-Score of miRNA Density by Chromosome')
        plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
        plt.tight_layout()
        
        # Save the individual plot for each organism
        plt.savefig(f'images/{organism}_zscore_plot.png')
        plt.close()

In [8]:
# Function to plot z-score for each organism as a subplot in one figure
def plot_all_z_scores(df_list, organisms):
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))  # 2 rows, 3 columns
    axes = axes.flatten()
    for i, (df, org) in enumerate(zip(df_list, organisms)):
        sns.barplot(data=df, x='Chromosome', y='z_score', hue='outlier', palette='coolwarm', ax=axes[i])
        axes[i].axhline(0, color='gray', linestyle='--')
        axes[i].axhline(3, color='red', linestyle='--')
        axes[i].axhline(-3, color='red', linestyle='--')
        axes[i].set_title(f'{org}: Z-Score of miRNA Density by Chromosome')
        axes[i].tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig('images/all_zscore_plots.png')  # Save combined figure
    plt.close()

In [9]:
# Function to plot the length of each chromosome and its number of miRNAs for comparison
# And save individual plots for each organism as well as a combined plot
def plot_chromosome_comparison(df_list, organisms):
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    fig.suptitle("Chromosome Length vs. miRNA Count", fontsize=16)  
    for i, (df, org) in enumerate(zip(df_list, organisms)):
        # Bar plot for chromosome length
        sns.barplot(x='Chromosome', y='Length (Mbp)', data=df, color='skyblue', ax=axes[i], label='Length (Mbp)')

        # Line plot for miRNA count
        sns.lineplot(x='Chromosome', y='Number of miRNA', data=df, color='red', marker='o', ax=axes[i], label='miRNA Count')

        axes[i].set_xticks(range(len(df['Chromosome'])))  # Set fixed tick positions
        axes[i].set_xticklabels(df['Chromosome'], rotation=90)
        axes[i].set_title(f'{org}')
        axes[i].set_xlabel('Chromosome')
        axes[i].set_ylabel('Value')
        axes[i].legend()

        # Save each individual figure
        plt.figure(figsize=(12, 6))
        sns.barplot(x='Chromosome', y='Length (Mbp)', data=df, color='skyblue', label='Length (Mbp)')
        sns.lineplot(x='Chromosome', y='Number of miRNA', data=df, color='red', marker='o', label='miRNA Count')
        plt.xticks(rotation=90)
        plt.legend()
        plt.title(f'{org}: Chromosome Length vs. miRNA Count')
        plt.xlabel('Chromosome')
        plt.ylabel('Value')
        plt.tight_layout()
        plt.savefig(f"images/{org}_comparison.png", dpi=300, bbox_inches='tight')
        plt.close()  # Close to prevent overlapping figures

    # Save the subplot figure
    plt.tight_layout()
    plt.savefig("images/all_organisms_comparison.png", dpi=300, bbox_inches='tight')
    plt.close()

# Run all functions for all organisms

In [11]:
# List of organism names
organisms = ['human', 'mouse', 'rat', 'fly', 'worm', 'arabidopsis']

# List to store dataframes for each organism
df_list = []

# Load data for each organism
for org in organisms:
    df = pd.read_excel('data/data.xlsx', sheet_name=org)  # Load the sheet corresponding to each organism
    # Convert base pairs to mega base pairs for chromosome length
    df = convert_to_megabase(df)
    # Calculate miRNA density to take chromosome length into account
    # For example, a chromosome might have lesser microRNAs only because it is smaller, and might actually have a higher miRNA density
    df = calculate_mirna_density(df)
    # Add each df to the df_list
    df_list.append(df)

# Normality tests
# Plot histograms and QQ plots for each organism as subplots in one figure and save figure
plot_all_histograms(df_list, organisms)
plot_all_qq(df_list, organisms)
# Shapiro-Wilk normality test
normality_results = run_shapiro_tests(df_list, 'miRNA_density')
# Print results
for organism, result in normality_results.items():
    print(f"{organism}: W-statistic = {result['W-statistic']}, p-value = {result['p-value']}") 
    # If p < 0.05, the miRNA density does not follow a normal distribution

# Calculate z-scores for each dataframe in the list
df_list = calculate_z_scores(df_list)

# Plot z-scores for each organism
plot_z_scores(df_list, organisms)

# Plot z-scores for all organisms as subplots in a figure
plot_all_z_scores(df_list, organisms)

# Plots to compare the length of each chromosome to its number of miRNAs
plot_chromosome_comparison(df_list, organisms)

human: W-statistic = 0.7316529382290032, p-value = 2.7191753919546344e-05
mouse: W-statistic = 0.9249711650668117, p-value = 0.10912690866764052
rat: W-statistic = 0.8602921699977656, p-value = 0.005190541945566848
fly: W-statistic = 0.9093860178437431, p-value = 0.4323693903085549
worm: W-statistic = 0.9830700129249585, p-value = 0.9657609809316614
arabidopsis: W-statistic = 0.7242536763210208, p-value = 0.01688807626189261
